In [116]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypdf import PdfReader
import re
from sentence_transformers import SentenceTransformer
import faiss
import pickle
import json
import os
from google import genai

from api_key import API_KEY

client = genai.Client(api_key=API_KEY)


In [117]:
def extract_text_from_pdf(pdf_path):
    """Extracts all text from a given PDF file."""
    reader = PdfReader(pdf_path)
    full_text = ""
    for page in reader.pages:
        full_text += page.extract_text() or "" 
    return full_text


file_path1 = 'documents/corep-own-funds-instructions.pdf'
file_path2 = 'documents/Reporting (CRR)_06-02-2026.pdf'
own_funds_instructions = extract_text_from_pdf(file_path1)
reporting_crr = extract_text_from_pdf(file_path2)


In [118]:
print(len(own_funds_instructions), len(reporting_crr))

586822 83341


In [119]:
data_list = list()
i =0
while (i < len(own_funds_instructions)):
    row = {'chunk_id': 'corep_'+str(int(i/1000)),'text':own_funds_instructions[i:i+1200]	,'source':'COREP_Annex_II'}
    i=i+1000
    data_list.append(row)

data1=pd.DataFrame(data_list)
def is_junk_chunk(text, min_len=200):
    if not isinstance(text, str):
        return True

    t = text.strip()

    if len(t) < min_len:
        return True

    no_space = re.sub(r"\s+", "", t)

    # mostly punctuation/dots
    if len(re.sub(r"[A-Za-z0-9]", "", no_space)) / len(no_space) > 0.85:
        return True

    # long dotted separator
    if re.search(r"\.{15,}", t):
        return True

    # very low alphabetic content
    alpha_chars = sum(c.isalpha() for c in t)
    if alpha_chars / len(t) < 0.15:
        return True

    return False


data1_clean = data1.copy()
data1_clean["is_junk"] = data1_clean["text"].apply(is_junk_chunk)

print("Before:", len(data1_clean))

data1_clean = data1_clean[data1_clean["is_junk"] == False] \
    .drop(columns=["is_junk"]) \
    .reset_index(drop=True)

print("After:", len(data1_clean))
# Separate COREP and PRA
corep_df = data1_clean[data1_clean["source"] == "COREP_Annex_II"].copy().reset_index(drop=True)


# Reassign sequential chunk_ids
corep_df["chunk_id"] = [f"corep_{i:04d}" for i in range(len(corep_df))]


# Merge back
data1 = pd.concat([corep_df], ignore_index=True)

# Check
data1



Before: 587
After: 562


,chunk_id,text,source
0,corep_0000,"erage; \n(b) group solvency, an overview of t...",COREP_Annex_II
1,corep_0001,rows and cells of the templates. Those numeri...,COREP_Annex_II
2,corep_0002,Abbreviations \n10. For the purposes of thi...,COREP_Annex_II
3,corep_0003,"s of certain types of undertakings, amending D...",COREP_Annex_II
4,corep_0004,\n11. The CA templates contain information abo...,COREP_Annex_II
...,...,...,...
557,corep_0557,"(row 0060), the part of NPEs secured by immov...",COREP_Annex_II
558,corep_0558,er Article 47c(6) CRR secured by immovable pro...,COREP_Annex_II
559,corep_0559,tion of the exposure as non-performing. \nEff...,COREP_Annex_II
560,corep_0560,"ints (a), (b), (c), (e) and (g) of Article 47c...",COREP_Annex_II


In [120]:
data_list_new = list()
i =0
while (i < len(reporting_crr)):
    row = {'chunk_id': 'pra_'+str(int(i/1000)),'text':reporting_crr[i:i+1200]	,'source':'PRA_RULEBOOK'}
    i=i+1000
    data_list_new.append(row)

data2=pd.DataFrame(data_list_new)
def is_junk_chunk(text, min_len=200):
    if not isinstance(text, str):
        return True

    t = text.strip()

    if len(t) < min_len:
        return True

    no_space = re.sub(r"\s+", "", t)

    # mostly punctuation/dots
    if len(re.sub(r"[A-Za-z0-9]", "", no_space)) / len(no_space) > 0.85:
        return True

    # long dotted separator
    if re.search(r"\.{15,}", t):
        return True

    # very low alphabetic content
    alpha_chars = sum(c.isalpha() for c in t)
    if alpha_chars / len(t) < 0.15:
        return True

    return False


data2_clean = data2.copy()
data2_clean["is_junk"] = data2_clean["text"].apply(is_junk_chunk)

print("Before:", len(data2_clean))

data2_clean = data2_clean[data2_clean["is_junk"] == False] \
    .drop(columns=["is_junk"]) \
    .reset_index(drop=True)

print("After:", len(data2_clean))
pra_df   = data2_clean[data2_clean["source"] == "PRA_RULEBOOK"].copy().reset_index(drop=True)

pra_df["chunk_id"]   = [f"pra_{i:04d}" for i in range(len(pra_df))]

# Merge back
data2 = pd.concat([ pra_df], ignore_index=True)

# Check
data2



Before: 84
After: 84


,chunk_id,text,source
0,pra_0000,Prudential Regulation Authority Rulebook\nPart...,PRA_RULEBOOK
1,pra_0001,s\n1.1 This Part applies to:\n(a) a firm that ...,PRA_RULEBOOK
2,pra_0002,5 and annexes X and XI of\nChapter 6.\n31/12/...,PRA_RULEBOOK
3,pra_0003,s on a consolidated basis\n2.4 A CRR consolida...,PRA_RULEBOOK
4,pra_0004,'consolidation situation' is defined in Artic...,PRA_RULEBOOK
...,...,...,...
79,pra_0079,.246 [Deleted.]\n01/09/2022\n2.247 [Deleted.]0...,PRA_RULEBOOK
80,pra_0080,22\n6.257 [Note: Provision left blank]\n01/09/...,PRA_RULEBOOK
81,pra_0081,/09/2022\n6.269 Annex XVI Template F 32.04 can...,PRA_RULEBOOK
82,pra_0082,found here\nO .\n01/09/2022\n6.279 Annex XVII...,PRA_RULEBOOK


In [121]:
data_final = pd.concat([data1,data2],ignore_index=True)
data_final

,chunk_id,text,source
0,corep_0000,"erage; \n(b) group solvency, an overview of t...",COREP_Annex_II
1,corep_0001,rows and cells of the templates. Those numeri...,COREP_Annex_II
2,corep_0002,Abbreviations \n10. For the purposes of thi...,COREP_Annex_II
3,corep_0003,"s of certain types of undertakings, amending D...",COREP_Annex_II
4,corep_0004,\n11. The CA templates contain information abo...,COREP_Annex_II
...,...,...,...
641,pra_0079,.246 [Deleted.]\n01/09/2022\n2.247 [Deleted.]0...,PRA_RULEBOOK
642,pra_0080,22\n6.257 [Note: Provision left blank]\n01/09/...,PRA_RULEBOOK
643,pra_0081,/09/2022\n6.269 Annex XVI Template F 32.04 can...,PRA_RULEBOOK
644,pra_0082,found here\nO .\n01/09/2022\n6.279 Annex XVII...,PRA_RULEBOOK


In [122]:

bad_patterns = r"\[Deleted\]|\[ Deleted \]|Provision left blank|can be found here|\[Deleted\.\]"

data_final = data_final[
    ~data_final["text"].str.contains(bad_patterns, regex=True, flags=re.IGNORECASE)
].copy()

data_final = data_final.reset_index(drop=True)
keep_keywords = r"COREP|own funds|CET1|Tier 1|Tier 2|capital requirements|CRR"

pra_useful = data_final[
    (data_final["source"] == "PRA_RULEBOOK") &
    (data_final["text"].str.contains(keep_keywords, regex=True, flags=re.IGNORECASE))
]

corep_all = data_final[data_final["source"] == "COREP_Annex_II"]

data_final = pd.concat([corep_all, pra_useful], ignore_index=True)
data_final = data_final.reset_index(drop=True)



In [123]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

embeddings = model.encode(
    data_final["text"].tolist(),
    show_progress_bar=True,normalize_embeddings=True
)

embeddings = np.array(embeddings).astype("float32")
data_final["embedding_text"] = list(embeddings)

# for i in data_final['text']:
#     data_final['text'] = model.encode(i)

Batches: 100%|██████████| 19/19 [00:01<00:00, 11.82it/s]


In [124]:
data_final

,chunk_id,text,source,embedding_text
0,corep_0000,"erage; \n(b) group solvency, an overview of t...",COREP_Annex_II,"[-0.011413075, 0.050157834, -0.06911405, -0.02..."
1,corep_0001,rows and cells of the templates. Those numeri...,COREP_Annex_II,"[-0.05913836, 0.056985375, -0.046240084, -0.06..."
2,corep_0002,Abbreviations \n10. For the purposes of thi...,COREP_Annex_II,"[-0.05379025, 0.00073226163, 0.024320433, -0.0..."
3,corep_0003,"s of certain types of undertakings, amending D...",COREP_Annex_II,"[-0.048888933, 0.013849427, 0.017130297, -0.01..."
4,corep_0004,\n11. The CA templates contain information abo...,COREP_Annex_II,"[-0.013768392, 0.037880532, 0.020612482, -0.03..."
...,...,...,...,...
589,pra_0041,tional liquidity monitoring metrics specified ...,PRA_RULEBOOK,"[0.017277189, -0.051744733, -0.06506463, -0.01..."
590,pra_0042,to report information on asset encumbrance in ...,PRA_RULEBOOK,"[-0.02410535, 0.074167915, -0.041559182, -0.04..."
591,pra_0044,ial holding\ncompanies and UK parent mixed fin...,PRA_RULEBOOK,"[-0.004714869, -0.03567785, 0.015623904, -0.01..."
592,pra_0045,"book, the following shall apply with regard to...",PRA_RULEBOOK,"[-0.0073001846, 0.019241735, -0.05826596, 0.00..."


In [125]:
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
index.add(embeddings)

faiss.write_index(index, "corep_faiss.index")


In [126]:
metadata = data_final[["chunk_id", "source", "text"]].to_dict(orient="records")

with open("corep_metadata.pkl", "wb") as f:
    pickle.dump(metadata, f)


In [127]:

def load_retrieval_system(index_path="corep_faiss.index",
                          metadata_path="corep_metadata.pkl",
                          embed_model_name="sentence-transformers/all-MiniLM-L6-v2"):
    
    model = SentenceTransformer(embed_model_name)
    index = faiss.read_index(index_path)

    with open(metadata_path, "rb") as f:
        metadata = pickle.load(f)

    return model, index, metadata


def retrieve_chunks(query, model, index, metadata, top_k=5):

    q_vec = model.encode([query], normalize_embeddings=True)
    q_vec = np.array(q_vec).astype("float32")

    distances, indices = index.search(q_vec, top_k)

    results = []
    for i, idx in enumerate(indices[0]):
        chunk_data = metadata[idx]

        results.append({
            "chunk_id": chunk_data["chunk_id"],
            "source": chunk_data["source"],
            "text": chunk_data["text"],
            "score": float(distances[0][i])
        })

    return results


def definition_print(query, model, index, metadata, top_k=5):
    retrieved = retrieve_chunks(query, model, index, metadata, top_k=top_k)

    for r in retrieved:
        print(r["chunk_id"], r["source"], r["score"])
        print(r["text"][:300])
        print("-----")


In [128]:

# ==============================
# CONFIG
# ==============================
MODEL_NAME = "models/gemini-3-flash-preview"
import re

ROW_EXTRACT = re.compile(r"(\d{4})")

def fix_row_codes(result_json):
    if result_json is None:
        return None

    # Fix populated_cells
    for cell in result_json.get("populated_cells", []):
        raw_row = str(cell.get("row", ""))
        match = ROW_EXTRACT.search(raw_row)
        if match:
            cell["row"] = match.group(1)

    # Fix audit_log
    for log in result_json.get("audit_log", []):
        raw_field = str(log.get("field", ""))
        match = ROW_EXTRACT.search(raw_field)
        if match:
            log["field"] = match.group(1)

    return result_json


# ==============================
# 1) INIT CLIENT
# ==============================
def init_gemini_client(api_key: str):
    """
    Initializes Gemini client using a key passed at runtime.
    """
    if not api_key or api_key.strip() == "":
        raise ValueError("❌ API key is missing or empty")

    return genai.Client(api_key=api_key)



client = init_gemini_client(API_KEY)



# ==============================
# 2) SCHEMA TEMPLATE
# ==============================
def get_schema_template():
    return """
{
  "template": "C 01.00",
  "currency": "GBP",
  "scenario_summary": "",
  "populated_cells": [
    {
      "row": "",
      "column": "",
      "item": "",
      "value": null,
      "unit": "GBP",
      "confidence": "",
      "source_chunk_ids": []
    }
  ],
  "validation_flags": [
    {
      "type": "missing_data|inconsistency|warning",
      "message": ""
    }
  ],
  "audit_log": [
    {
      "field": "",
      "value": null,
      "justification": "",
      "source_chunk_ids": []
    }
  ]
}
"""


# ==============================
# 3) CONTEXT BUILDER
# ==============================
def build_context(retrieved_chunks):
    context = ""
    for chunk in retrieved_chunks:
        context += f"[{chunk['chunk_id']} | {chunk['source']}]\n"
        context += chunk["text"] + "\n\n"
    return context


# ==============================
# 4) PROMPT BUILDER
# ==============================
def build_prompt(question, scenario, retrieved_chunks, schema_template):
    context = build_context(retrieved_chunks)

    prompt = f"""
You are a PRA COREP regulatory reporting assistant.

TASK:
Generate a structured COREP Own Funds reporting output for Template C 01.00.

RULES:
- Use ONLY the scenario and the retrieved regulatory context.
- Output MUST be valid JSON only (no markdown, no explanation).
- Every populated cell MUST include source_chunk_ids.
- If a value cannot be derived, set it to null and add a validation flag.
- Tier 1 = CET1 + AT1 if values exist.
- Own Funds = Tier 1 + Tier 2 if values exist.
- IMPORTANT: All numeric values must be in FULL GBP units (not millions).
  Example: 540 million GBP must be written as 540000000.
- In justification, always write values in full GBP format (e.g., 480000000 not 480m).


Scenario:
{scenario}

Question:
{question}

Retrieved regulatory context:
{context}

Return JSON strictly following this schema:
{schema_template}

IMPORTANT:
Output must start with {{ and end with }} only.
"""
    return prompt

def fix_million_values(result_json):
    """
    If values look like millions (ex: 480 instead of 480000000),
    convert them into full GBP units.
    """
    for cell in result_json.get("populated_cells", []):
        val = cell.get("value")

        if isinstance(val, (int, float)):
            # If value is too small, assume it's in millions
            if val < 1000000:
                cell["value"] = int(val * 1_000_000)

    for log in result_json.get("audit_log", []):
        val = log.get("value")
        if isinstance(val, (int, float)):
            if val < 1000000:
                log["value"] = int(val * 1_000_000)

    return result_json



# ==============================
# 5) GEMINI CALLER
# ==============================
def call_gemini(client, prompt, model_name=MODEL_NAME):
    response = client.models.generate_content(
        model=model_name,
        contents=prompt
    )
    return response.text


# ==============================
# 6) JSON PARSER
# ==============================
def parse_json_output(llm_output):
    llm_output = re.sub(r"```json|```", "", llm_output).strip()

    try:
        return json.loads(llm_output)
    except:
        print("❌ JSON Parsing Failed. Raw output below:\n")
        print(llm_output)
        return None


# ==============================
# 7) JSON GENERATION PIPELINE
# ==============================
def generate_corep_json(client, question, scenario, retrieved_chunks):
    schema_template = get_schema_template()

    prompt = build_prompt(
        question=question,
        scenario=scenario,
        retrieved_chunks=retrieved_chunks,
        schema_template=schema_template
    )

    llm_output = call_gemini(client, prompt)

    structured_json = parse_json_output(llm_output)

    return structured_json


# ==============================
# 8) VALIDATION FUNCTION
# ==============================
# ==============================
# UPDATED SCHEMA + VALIDATION
# MATCHING COREP ANNEX ROW CODES
# ==============================

def get_schema_template():
    return """
{
  "template": "C 01.00",
  "currency": "GBP",
  "scenario_summary": "",
  "populated_cells": [
    {
      "row": "0010",
      "column": "0010",
      "item": "OWN FUNDS",
      "value": null,
      "unit": "GBP",
      "confidence": "",
      "source_chunk_ids": []
    },
    {
      "row": "0015",
      "column": "0010",
      "item": "TIER 1 CAPITAL",
      "value": null,
      "unit": "GBP",
      "confidence": "",
      "source_chunk_ids": []
    },
    {
      "row": "0020",
      "column": "0010",
      "item": "COMMON EQUITY TIER 1 CAPITAL",
      "value": null,
      "unit": "GBP",
      "confidence": "",
      "source_chunk_ids": []
    },
    {
      "row": "0030",
      "column": "0010",
      "item": "ADDITIONAL TIER 1 CAPITAL",
      "value": null,
      "unit": "GBP",
      "confidence": "",
      "source_chunk_ids": []
    },
    {
      "row": "0040",
      "column": "0010",
      "item": "TIER 2 CAPITAL",
      "value": null,
      "unit": "GBP",
      "confidence": "",
      "source_chunk_ids": []
    }
  ],
  "validation_flags": [],
  "audit_log": [
    {
      "field": "0010",
      "value": null,
      "justification": "",
      "source_chunk_ids": []
    },
    {
      "field": "0015",
      "value": null,
      "justification": "",
      "source_chunk_ids": []
    },
    {
      "field": "0020",
      "value": null,
      "justification": "",
      "source_chunk_ids": []
    }
  ]
}
"""


# ==============================
# STRICT PROMPT ROW RULES
# ==============================
def get_row_mapping_text():
    return """
STRICT ROW MAPPING FOR TEMPLATE C 01.00:
- 0010 = Own Funds
- 0015 = Tier 1 Capital
- 0020 = Common Equity Tier 1 (CET1)
- 0030 = Additional Tier 1 (AT1)
- 0040 = Tier 2 Capital

RULE:
Tier 1 (0015) = CET1 (0020) + AT1 (0030)
Own Funds (0010) = Tier 1 (0015) + Tier 2 (0040)
"""


# ==============================
# UPDATED PROMPT BUILDER
# ==============================
def build_prompt(question, scenario, retrieved_chunks, schema_template):
    context = build_context(retrieved_chunks)

    prompt = f"""
You are a PRA COREP regulatory reporting assistant.

TASK:
Populate COREP Own Funds Template C 01.00.

STRICT RULES:
- Output MUST be valid JSON only.
- DO NOT output markdown.
- DO NOT invent new row numbers.
- Use ONLY these row codes: 0010, 0015, 0020, 0030, 0040.
- Every populated cell MUST contain source_chunk_ids.
- If a value cannot be derived, set it to null and add a validation flag.

{get_row_mapping_text()}

Scenario:
{scenario}

Question:
{question}

Retrieved regulatory context:
{context}

Return JSON strictly following this schema:
{schema_template}
"""
    return prompt


# ==============================
# UPDATED VALIDATOR
# ==============================
def validate_corep_output(result_json):
    flags = []

    row_map = {cell["row"]: cell["value"] for cell in result_json.get("populated_cells", [])}

    own_funds = row_map.get("0010")
    tier1 = row_map.get("0015")
    cet1 = row_map.get("0020")
    at1 = row_map.get("0030")
    tier2 = row_map.get("0040")

    # Required rows check
    required_rows = ["0010", "0015", "0020", "0030", "0040"]
    for r in required_rows:
        if r not in row_map:
            flags.append({
                "type": "missing_data",
                "message": f"Missing required row {r}"
            })

    # Tier 1 check
    if cet1 is not None and at1 is not None and tier1 is not None:
        expected_tier1 = cet1 + at1
        if tier1 != expected_tier1:
            flags.append({
                "type": "inconsistency",
                "message": f"Tier 1 mismatch: expected {expected_tier1}, got {tier1}"
            })

    # Own funds check
    if tier1 is not None and tier2 is not None and own_funds is not None:
        expected_own_funds = tier1 + tier2
        if own_funds != expected_own_funds:
            flags.append({
                "type": "inconsistency",
                "message": f"Own Funds mismatch: expected {expected_own_funds}, got {own_funds}"
            })

    return flags



# ==============================
# 9) AUDIT LOG EVIDENCE ATTACHER
# ==============================
def attach_evidence(audit_log, retrieved_chunks, snippet_len=300):
    chunk_map = {c["chunk_id"]: c["text"] for c in retrieved_chunks}

    for entry in audit_log:
        evidence = []
        for cid in entry.get("source_chunk_ids", []):
            if cid in chunk_map:
                evidence.append(chunk_map[cid][:snippet_len])
        entry["evidence_snippets"] = evidence

    return audit_log


# ==============================
# 10) AUDIT LOG PRINTER
# ==============================
def print_audit_log(audit_log):
    for entry in audit_log:
        print("FIELD:", entry.get("field"))
        print("VALUE:", entry.get("value"))
        print("JUSTIFICATION:", entry.get("justification"))
        print("SOURCE CHUNKS:", entry.get("source_chunk_ids"))
        if "evidence_snippets" in entry:
            print("EVIDENCE:", entry["evidence_snippets"])
        print("-----")
def merge_unique_chunks(chunks1, chunks2):
    seen = set()
    merged = []

    for c in chunks1 + chunks2:
        if c["chunk_id"] not in seen:
            merged.append(c)
            seen.add(c["chunk_id"])

    return merged


def retrieve_corep_context(question, scenario, model, index, metadata, top_k=8):
    """
    Dual retrieval strategy:
    1) retrieve using user question
    2) retrieve using COREP row-mapping query
    Merge results for best context.
    """

    # Query 1: pure user question (semantic)
    q1 = question.strip()

    # Query 2: template + row mapping query (forces correct row context)
    q2 = f"""
COREP Own Funds Template C 01.00 instructions
CET1 row 0530
AT1 row 0540
Tier 1 row 0740
Tier 2 row 0750
Own Funds row 0980
Intangible assets deduction row 0300
Deferred tax assets deduction row 0340
""".strip()

    retrieved_q1 = retrieve_chunks(q1, model, index, metadata, top_k=top_k)
    retrieved_q2 = retrieve_chunks(q2, model, index, metadata, top_k=top_k)

    final_retrieved = merge_unique_chunks(retrieved_q1, retrieved_q2)

    # sort by score descending (higher similarity first)
    final_retrieved = sorted(final_retrieved, key=lambda x: x["score"], reverse=True)

    return final_retrieved[:top_k*2]   # keep more context

# ==============================
# 11) FINAL DEPLOYMENT FUNCTION
# ==============================
def corep_assistant(client, question, scenario, model, index, metadata, top_k=8):

    retrieved_chunks = retrieve_corep_context(
        question=question,
        scenario=scenario,
        model=model,
        index=index,
        metadata=metadata,
        top_k=top_k
    )

    result_json = generate_corep_json(
        client=client,
        question=question,
        scenario=scenario,
        retrieved_chunks=retrieved_chunks
    )

    if result_json is None:
        return {"error": "LLM output invalid JSON"}

    # validate
    result_json["validation_flags"] = validate_corep_output(result_json)
    result_json = fix_row_codes(result_json)
    result_json = fix_million_values(result_json)


    # attach evidence snippets
    if "audit_log" in result_json:
        result_json["audit_log"] = attach_evidence(result_json["audit_log"], retrieved_chunks)

    return {
        "structured_json": result_json,
        "audit_log": result_json.get("audit_log", []),
        "retrieved_chunks": retrieved_chunks
    }


    

# ==============================
# 12) EXAMPLE USAGE (RUN THIS)
# ==============================
# client = init_gemini_client()
#
# scenario = """
# CET1 = 540 million GBP
# AT1 = 100 million GBP
# Tier 2 = 80 million GBP
# Intangible assets deduction = 40 million GBP
# Deferred tax assets deduction = 20 million GBP
# """
#
# question = "How should Tier 1 capital and Total Own Funds be reported in COREP template C 01.00?"
#
# output = corep_assistant(client, question, scenario, top_k=8)
#
# print(output["structured_json"]["validation_flags"])
# print_audit_log(output["audit_log"])
# print(json.dumps(output["audit_log"], indent=4))


In [129]:
def json_to_corep_table(result_json):
    """
    Converts structured JSON output into a human-readable COREP template extract.
    Returns a Pandas DataFrame.
    """
    if result_json is None:
        return None

    populated_cells = result_json.get("populated_cells", [])

    if len(populated_cells) == 0:
        print("⚠️ No populated cells found in JSON.")
        return pd.DataFrame()

    df = pd.DataFrame(populated_cells)

    # Ensure consistent ordering of columns
    expected_cols = ["row", "column", "item", "value", "unit", "confidence", "source_chunk_ids"]
    for col in expected_cols:
        if col not in df.columns:
            df[col] = None

    df = df[expected_cols]

    # Convert chunk ids list into readable string
    df["source_chunk_ids"] = df["source_chunk_ids"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

    return df
def print_corep_template(df):
    if df is None or df.empty:
        print("⚠️ COREP template table is empty.")
        return

    print("\n📌 COREP Template Extract (C 01.00)\n")
    display(df)


In [130]:
scenario = """
CET1 = 540 million GBP
AT1 = 100 million GBP
Tier 2 = 80 million GBP
Intangible assets deduction = 40 million GBP
Deferred tax assets deduction = 20 million GBP
"""

question = "How should Additional Tier 1 capital and Total Own Funds be reported in COREP template C 01.00?"

query = f"Scenario:\n{scenario}\n\nQuestion:\n{question}"

definition_print(query, model, index, metadata, top_k=5)

output = corep_assistant(client, question, scenario, model, index, metadata, top_k=8)

print(output["structured_json"]["validation_flags"])
print_audit_log(output["audit_log"])
print(json.dumps(output["audit_log"], indent=4))
result_json = output["structured_json"]

corep_df = json_to_corep_table(result_json)

print_corep_template(corep_df)

corep_0103 COREP_Annex_II 0.6224305629730225
 5.2 provides further details on the calculation of those grandfathered 
instruments which do not constitute state aid.   
18. Institutions shall report in the first four columns the adjustments to Common 
Equity Tier 1 capital, Additional Tier 1 capital a nd Tier 2 capital as well as the 
amount to
-----
corep_0140 COREP_Annex_II 0.6013277769088745
VE GOODWILL  
0360- 
0400  CONSOLIDATED OWN FUNDS  
Article 18 CRR  
The amount to be reported as “CONSOLIDATED OWN FUNDS” shall be the amount 
as derived from the balance sheet, excluding any fund brought in by other group entities.  
0360  CONSOLIDATED OWN FUNDS  
0370  OF WHICH: COMMON EQUITY TIE
-----
corep_0139 COREP_Annex_II 0.5999736189842224
NDS INCLUDED IN CONSOLIDATED OWN FUNDS  
Article 87 CRR  
0310  QUALIFYING TIER 1 INSTRUMENTS INCLUDED IN CONSOLIDATED TIER 1 
CAPITAL  
Article 85 CRR  
Effective from 1 January 20220320  MINORITY INTERESTS INCLUDED IN CONSOLIDATED COMMON  
EQUITY TI

,row,column,item,value,unit,confidence,source_chunk_ids
0,0010,0010,OWN FUNDS,660000000,GBP,High,corep_0007
1,0015,0010,TIER 1 CAPITAL,580000000,GBP,High,corep_0007
2,0020,0010,COMMON EQUITY TIER 1 CAPITAL,480000000,GBP,High,"corep_0007, corep_0023"
3,0030,0010,ADDITIONAL TIER 1 CAPITAL,100000000,GBP,High,corep_0140
4,0040,0010,TIER 2 CAPITAL,80000000,GBP,High,corep_0121
